# CSVS to Hepfile

Imagine we have a database like structure with multiple csvs all connected by a common ID. See the household exmaple here: https://hepfile.readthedocs.io/en/latest/fundamentals.html#a-toy-example 

In [ ]:
import os, glob
import pandas as pd
import numpy as np
import awkward as ak
import hepfile as hf

In [ ]:
datapath = os.path.join('/', 'home', 'nfranz', 'research', 'hepfile', 'examples', '*.csv')
files = glob.glob(datapath)
common_key = 'Household ID'
group_names = ['Residences', 'People', 'Vehicles']

if group_names is None:
    group_names = [os.path.split(file) for file in files]

for_ak = {}
for f, group_name in zip(files, group_names):
    
    csv = pd.read_csv(f)
    
    groups = csv.groupby(common_key)
    split_groups = []
    for item in groups.groups:
        split_groups.append(groups.get_group(item))
    
    subdict = {}
    for grouping in split_groups:
        for colname in grouping.columns:
            if colname in subdict.keys():
                subdict[colname].append(list(grouping[colname].values))
            else:
                subdict[colname] = [list(grouping[colname].values)]
                
    for key in subdict.keys():
        subdict[key] = ak.Array(subdict[key])
        
    
    for_ak[group_name] = subdict
for_ak

In [ ]:
hf.awkward_tools.awkward_to_hepfile(for_ak, 'testing_csv_translation.h5')